In [1]:
import cv2
import matplotlib.pyplot as plt
import cvlib as cv
from cvlib.object_detection import draw_bbox
import pyautogui
from PIL import ImageOps, Image
import numpy as np
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'

In [4]:
cnt_cars = 0
idi = []
idi.append(cnt_cars)
idi[cnt_cars] = 0
cbox = 0 
xcoord = []
ccoox0 = []
ccoox0.append(0)
ccoox1 = []
ccoox1.append(0)
ccoox2 = []
ccoox2.append(0)
ccoox3 = []
ccoox3.append(0)
fstr = []
a=0
b=0
num_frames = 0

cap = cv2.VideoCapture("VID20220614080007.mp4")
# while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret == True:
#         num_frames += 1
#         frame = cv2.cvtColor(frame[500:1040, 250:1800], cv2.COLOR_BGR2RGB)
#         im = np.array(frame)
#         # bbox, label, conf = cv.detect_common_objects(im)
#         obj = cv.detect_common_objects(im)
#         for i in range(len(obj[1])):
#             if obj[1][i] == 'car' or obj[1][i] == 'truck' or obj[1][i] == 'bus':
#                 ccoox0.append(obj[0][i][0])
#                 ccoox1.append(obj[0][i][1])
#                 ccoox2.append(obj[0][i][2])
#                 ccoox3.append(obj[0][i][3])
#     break
    
       
while(cap.isOpened()):
    ret, aframe = cap.read()
    cframe = aframe
    if ret == True:
        num_frames += 1
        if(num_frames==1):
            previousframe = cv2.cvtColor(cframe[500:850, 250:1800], cv2.COLOR_BGR2RGB)
        if(num_frames>=2):
            currentframe = cv2.cvtColor(cframe[500:850, 250:1800], cv2.COLOR_BGR2RGB)   
            frame = cv2.subtract(previousframe,currentframe)
        
        previousframe = cv2.cvtColor(cframe[500:850, 250:1800], cv2.COLOR_BGR2RGB)
    # img2 = pyautogui.screenshot(region=(250, 500, 1250, 500))
    
    # while img != img2:
        # img = pyautogui.screenshot(region=(250, 500, 1250, 500))
        # img2 = img
        # im = np.array(img)
        im = np.array(frame)
        # bbox, label, conf = cv.detect_common_objects(im)
        obj = cv.detect_common_objects(im)
        for i in range(len(obj[1])):
            if obj[1][i] == 'car' or obj[1][i] == 'truck' or obj[1][i] == 'bus':
                parado = 0
                for j in range(len(ccoox0)):
                    if len(ccoox0) >= len(obj[1]):
                        rm = 1
                    else:
                        rm = 0
                    difx0 = abs(abs(ccoox0[j]) - abs(obj[0][i][0]))
                    difx1 = abs(abs(ccoox1[j]) - abs(obj[0][i][1]))
                    difx2 = abs(abs(ccoox2[j]) - abs(obj[0][i][2]))
                    difx3 = abs(abs(ccoox3[j]) - abs(obj[0][i][3]))
                    x0 = obj[0][i][0]
                    y0 = obj[0][i][1]
                    x1 = obj[0][i][2]
                    y1 = obj[0][i][3]
                    if difx0 < 100 and difx1 < 30 and difx2 < 100 and difx3 < 30:
                    # if difx0 < 40 and difx1 < 40:
                        a=1
                        mit = int(y0 + ((y1 - y0)/2))
                        if mit > 300 and mit < 350:
                            im_crop = im[mit:y1, x0:x1]
                            gray = cv2.cvtColor(im_crop, cv2.COLOR_BGR2GRAY)
                            gray = cv2.blur(gray,(1,1))
                            gray = cv2.bilateralFilter(gray, 8, 17, 17)
                            canny = cv2.Canny(gray,50,120)
                            cnts,_ = cv2.findContours(canny,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
                            for c in cnts:
                                area = cv2.contourArea(c)
                                x,y,w,h = cv2.boundingRect(c)
                                epsilon = 0.018*cv2.arcLength(c,True)
                                approx = cv2.approxPolyDP(c,epsilon,True)
                                if len(approx)==4 and area>900:
                                    aspect_ratio = float(w)/h
                                    if aspect_ratio>3.4 and aspect_ratio<6:
                                        placa = gray[y:y+h,x:x+w]
                                        text = pytesseract.image_to_string(placa,config='--psm 11')
                                        if len(text) >= 4:
                                            fstr.append(text + str(num_frames))
                                            print('PLACA: ',text)
                        if (ccoox1[j] < 330 and y0 >= 330 and difx1 > 0) or (y0 == 330 and ccoox1[j] >= 330):
                            cnt_cars += 1                 
                if a == 1 and obj[0][i][1] <331:
                    ccoox0[i] = obj[0][i][0]
                    ccoox1[i] = obj[0][i][1]
                    ccoox2[i] = obj[0][i][2]
                    ccoox3[i] = obj[0][i][3]
                    a = 0
                elif rm == 1 and obj[0][i][1] <331:
                    ccoox0[i] = obj[0][i][0]
                    ccoox1[i] = obj[0][i][1]
                    ccoox2[i] = obj[0][i][2]
                    ccoox3[i] = obj[0][i][3]
                    a = 0  
                elif obj[0][i][1] <331:
                    ccoox0.append(obj[0][i][0])
                    ccoox1.append(obj[0][i][1])
                    ccoox2.append(obj[0][i][2])
                    ccoox3.append(obj[0][i][3])
                    a = 0
 
        print(ccoox0, ccoox1, ccoox2, ccoox3)             
        print('Nº de coches: ', cnt_cars)
        print("MATRICULAS: ",fstr)
        output_image = draw_bbox(im, obj[0], obj[1], obj[2])
        # recta = cv2.rectangle(im, (0,330), (1900, 330), (255,0,0), 1) 
        cv2.imshow('Detection', output_image)
        # cv2.imshow('Detection', recta)
  
       
    if cv2.waitKey(1) == ord('q'): # si el usuario presiona q paramos de grabar.

        break
        

cv2.destroyAllWindows() # cerrar la ventana

[0] [42] [328] [312]
Nº de coches:  0
MATRICULAS:  []
[0] [42] [328] [312]
Nº de coches:  0
MATRICULAS:  []
[0] [42] [328] [312]
Nº de coches:  0
MATRICULAS:  []
[0] [42] [328] [312]
Nº de coches:  0
MATRICULAS:  []
[91] [64] [530] [268]
Nº de coches:  0
MATRICULAS:  []
[43] [62] [529] [273]
Nº de coches:  0
MATRICULAS:  []
[18] [62] [526] [271]
Nº de coches:  0
MATRICULAS:  []
[-1] [61] [499] [282]
Nº de coches:  0
MATRICULAS:  []
[-1] [61] [499] [282]
Nº de coches:  0
MATRICULAS:  []
[-3] [61] [489] [291]
Nº de coches:  0
MATRICULAS:  []
[-6] [57] [478] [293]
Nº de coches:  0
MATRICULAS:  []
[0] [54] [462] [296]
Nº de coches:  0
MATRICULAS:  []
[-3] [55] [453] [302]
Nº de coches:  0
MATRICULAS:  []
[-4] [55] [432] [304]
Nº de coches:  0
MATRICULAS:  []
[-5] [55] [419] [309]
Nº de coches:  0
MATRICULAS:  []
[-3] [53] [405] [310]
Nº de coches:  0
MATRICULAS:  []
[0] [67] [389] [323]
Nº de coches:  0
MATRICULAS:  []
[-1] [48] [361] [343]
Nº de coches:  0
MATRICULAS:  []
[-1] [49] [341] 